<a href="https://colab.research.google.com/github/WILLIANSCASACOLA/Listar_Arquivos_python/blob/main/Spark_mao_no_codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
#Instalar o java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip uninstall pyspark -y
!pip install pyspark==3.1.2

Found existing installation: pyspark 3.1.2
Uninstalling pyspark-3.1.2:
  Successfully uninstalled pyspark-3.1.2
  Using cached pyspark-3.1.2-py2.py3-none-any.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.5 requires pyspark[connect]~=3.5.1, but you have pyspark 3.1.2 which is incompatible.


In [1]:
#Baixar a versão mais recente do spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [2]:
#Deszipar o spark
!tar xf /content/spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
#Criar varoáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"


In [4]:
# Instalar a lib findspark que ajuda a localizar o Spark no sistema e
# importá-lo como uma biblioteca regular
!pip install -q findspark

In [5]:
import findspark
findspark.init()

In [6]:
#Criar spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("sparkColab")\
        .getOrCreate()

In [7]:
#Importar o dataset
from google.colab import files
arquivo = files.upload()

In [8]:
#Criar datafreme
df = spark.read.csv("dataset.csv", header=True, inferSchema=True)

In [9]:
#Mostrar os tipos dos dados
df.printSchema()

root
 |-- ID_Pedido: string (nullable = true)
 |-- Data_Pedido: string (nullable = true)
 |-- ID_Cliente: string (nullable = true)
 |-- Segmento: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Regiao: string (nullable = true)
 |-- ID_Produto: string (nullable = true)
 |-- Categoria: string (nullable = true)
 |-- Nome_Produto: string (nullable = true)
 |-- Valor_Venda: string (nullable = true)
 |-- Quantidade: string (nullable = true)



In [10]:
#Mostrar os dados do df
df.show()

+--------------+-----------+----------+-----------+-------------+-------+---------------+---------------+--------------------+-----------+----------+
|     ID_Pedido|Data_Pedido|ID_Cliente|   Segmento|         Pais| Regiao|     ID_Produto|      Categoria|        Nome_Produto|Valor_Venda|Quantidade|
+--------------+-----------+----------+-----------+-------------+-------+---------------+---------------+--------------------+-----------+----------+
|CA-2016-152156| 2016-11-08|  CG-12520|   Consumer|United States|  South|FUR-BO-10001798|      Furniture|Bush Somerset Col...|     261.96|      null|
|CA-2016-152156| 2016-11-08|  CG-12520|   Consumer|United States|  South|FUR-CH-10000454|      Furniture|Hon Deluxe Fabric...|     731.94|      null|
|CA-2016-138688| 2016-06-12|  DV-13045|  Corporate|United States|   West|OFF-LA-10000240|Office Supplies|Self-Adhesive Add...|      14.62|         2|
|US-2015-108966| 2015-10-11|  SO-20335|   Consumer|United States|  South|FUR-TA-10000577|      Furni

In [11]:
#Selecionar colunas:
df1 = df.select("Segmento", "Pais","Regiao","Nome_Produto","Valor_Venda","Quantidade")

In [12]:
df1.show()

+-----------+-------------+-------+--------------------+-----------+----------+
|   Segmento|         Pais| Regiao|        Nome_Produto|Valor_Venda|Quantidade|
+-----------+-------------+-------+--------------------+-----------+----------+
|   Consumer|United States|  South|Bush Somerset Col...|     261.96|      null|
|   Consumer|United States|  South|Hon Deluxe Fabric...|     731.94|      null|
|  Corporate|United States|   West|Self-Adhesive Add...|      14.62|         2|
|   Consumer|United States|  South|Bretford CR4500 S...|   957.5775|         5|
|   Consumer|United States|  South|Eldon Fold 'N Rol...|     22.368|         2|
|   Consumer|United States|   West|Eldon Expressions...|      48.86|         7|
|   Consumer|United States|   West|          Newell 322|       7.28|         4|
|   Consumer|United States|   West|Mitel 5320 IP Pho...|    907.152|         6|
|   Consumer|United States|   West|DXL Angle-View Bi...|     18.504|         3|
|   Consumer|United States|   West|Belki

In [13]:
#Mudar o tipo das colunas: "Valor_Venda" and "Quantidade" para "Integer"
from pyspark.sql.types import IntegerType
df1 = df1.withColumn("Valor_Venda", df1["Valor_Venda"].cast(IntegerType()))
df1 = df1.withColumn("Quantidade", df1["Quantidade"].cast(IntegerType()))

In [14]:
#Mostrar schema
df1.printSchema()

root
 |-- Segmento: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Regiao: string (nullable = true)
 |-- Nome_Produto: string (nullable = true)
 |-- Valor_Venda: integer (nullable = true)
 |-- Quantidade: integer (nullable = true)



In [15]:
df1.show()

+-----------+-------------+-------+--------------------+-----------+----------+
|   Segmento|         Pais| Regiao|        Nome_Produto|Valor_Venda|Quantidade|
+-----------+-------------+-------+--------------------+-----------+----------+
|   Consumer|United States|  South|Bush Somerset Col...|        261|      null|
|   Consumer|United States|  South|Hon Deluxe Fabric...|        731|      null|
|  Corporate|United States|   West|Self-Adhesive Add...|         14|         2|
|   Consumer|United States|  South|Bretford CR4500 S...|        957|         5|
|   Consumer|United States|  South|Eldon Fold 'N Rol...|         22|         2|
|   Consumer|United States|   West|Eldon Expressions...|         48|         7|
|   Consumer|United States|   West|          Newell 322|          7|         4|
|   Consumer|United States|   West|Mitel 5320 IP Pho...|        907|         6|
|   Consumer|United States|   West|DXL Angle-View Bi...|         18|         3|
|   Consumer|United States|   West|Belki

In [16]:
#Deletar a coluna no df1 Quantidade_ se ela existir
df1 = df1.drop("Quantidade_")

In [17]:
df1.show()

+-----------+-------------+-------+--------------------+-----------+----------+
|   Segmento|         Pais| Regiao|        Nome_Produto|Valor_Venda|Quantidade|
+-----------+-------------+-------+--------------------+-----------+----------+
|   Consumer|United States|  South|Bush Somerset Col...|        261|      null|
|   Consumer|United States|  South|Hon Deluxe Fabric...|        731|      null|
|  Corporate|United States|   West|Self-Adhesive Add...|         14|         2|
|   Consumer|United States|  South|Bretford CR4500 S...|        957|         5|
|   Consumer|United States|  South|Eldon Fold 'N Rol...|         22|         2|
|   Consumer|United States|   West|Eldon Expressions...|         48|         7|
|   Consumer|United States|   West|          Newell 322|          7|         4|
|   Consumer|United States|   West|Mitel 5320 IP Pho...|        907|         6|
|   Consumer|United States|   West|DXL Angle-View Bi...|         18|         3|
|   Consumer|United States|   West|Belki

In [18]:
#Filtrar o df1 e classificar pelo "Valor_Venda"
df1.filter("Valor_Venda > 1000").orderBy("Valor_Venda").show()

+-----------+-------------+-------+--------------------+-----------+----------+
|   Segmento|         Pais| Regiao|        Nome_Produto|Valor_Venda|Quantidade|
+-----------+-------------+-------+--------------------+-----------+----------+
|Home Office|United States|Central|   Samsung Galaxy S4|       1001|         2|
|   Consumer|United States|   West|   Samsung Galaxy S4|       1001|         2|
|   Consumer|United States|   West|Office Impression...|       1003|         6|
|Home Office|United States|   West|Hon 2111 Invitati...|       1004|         6|
|   Consumer|United States|   West|Bevis Round Confe...|       1004|         7|
|   Consumer|United States|   East|Smead Adjustable ...|       1006|         3|
|Home Office|United States|   East|Luxo Professional...|       1007|         6|
|   Consumer|United States|   West|Polycom VVX 310 V...|       1007|         7|
|Home Office|United States|Central|Canon Color Image...|       1007|         3|
|   Consumer|United States|  South|Plant

In [19]:
#Pegar o valor Min e Max da coluna "Valor_Vendas"
from pyspark.sql.functions import min, max
df1.select(min("Valor_Venda").alias('Min_Valor'),\
           max("Valor_Venda").alias('Max_Valor'))\
           .show()

+---------+---------+
|Min_Valor|Max_Valor|
+---------+---------+
|        0|    22638|
+---------+---------+



In [20]:
Valor_maximo = df1.select(max("Valor_Venda").alias('Max_Valor'))

In [21]:
Valor_maximo.show()

+---------+
|Max_Valor|
+---------+
|    22638|
+---------+



In [25]:
#Filtra a linha inteira onde "Valor_Venda" seja igual ao "Valor_maximo"
linha_max = df1.filter(df1.Valor_Venda == Valor_maximo.collect()[0][0])

In [26]:
linha_max.show()

+-----------+-------------+------+--------------------+-----------+----------+
|   Segmento|         Pais|Regiao|        Nome_Produto|Valor_Venda|Quantidade|
+-----------+-------------+------+--------------------+-----------+----------+
|Home Office|United States| South|Cisco TelePresenc...|      22638|         6|
+-----------+-------------+------+--------------------+-----------+----------+



In [31]:
#Data de hoje
from datetime import date
hoje = date.today()
print(hoje)

2025-06-08


In [37]:
#Exportar "linha_max" para um csv
linha_max.toPandas().to_csv(f'linha_max_{hoje}.csv')
print(f"Dados exportados com êxito 'linha_max_{hoje}.csv'")

Dados exportados com êxito 'linha_max_2025-06-08.csv'


In [64]:
#Filtrar df1 onde coluna "Nome_Produto" contém a palavra: "Cisco"
from pyspark.sql.functions import col
df1_filtrado = df1.filter(col("Nome_Produto").like("%Cisco%"))


In [65]:
df1_filtrado.show()

+-----------+-------------+-------+--------------------+-----------+----------+
|   Segmento|         Pais| Regiao|        Nome_Produto|Valor_Venda|Quantidade|
+-----------+-------------+-------+--------------------+-----------+----------+
|   Consumer|United States|   West|Cisco SPA 501G IP...|        213|         3|
|  Corporate|United States|   East|Cisco 9971 IP Vid...|       1188|         9|
|   Consumer|United States|Central|Cisco Small Busin...|        328|         4|
|  Corporate|United States|   West|Cisco SPA525G2 IP...|         95|         6|
|   Consumer|United States|   West|        Cisco SPA301|        374|         3|
|  Corporate|United States|   East|Cisco 8x8 Inc. 67...|        728|         9|
|   Consumer|United States|  South|Cisco Unified IP ...|       1363|         5|
|Home Office|United States|   East|        Cisco SPA301|        280|         3|
|   Consumer|United States|   East|Cisco SPA525G2 5-...|       1379|         8|
|  Corporate|United States|   East|     

In [59]:
#Contar quantidade de linha no "df1_filtrado"
df1_filtrado.count()

5

In [60]:
#Incluir uma coluna chamada "DtIngest" com a data de hoje no "df1_filtrado"
from pyspark.sql.functions import lit
df1_filtrado = df1_filtrado.withColumn("DtIngest", lit(hoje))

#Criar uma nova coluna chamada "Valor_Total" multimpliando "Valor_Venda" x "Quantidade"
df1_filtrado = df1_filtrado.withColumn("Valor_Total", df1_filtrado.Valor_Venda * df1_filtrado.Quantidade)


In [61]:
df1_filtrado.show()

+---------+-------------+-------+-------------+-----------+----------+----------+-----------+
| Segmento|         Pais| Regiao| Nome_Produto|Valor_Venda|Quantidade|  DtIngest|Valor_Total|
+---------+-------------+-------+-------------+-----------+----------+----------+-----------+
|Corporate|United States|   East|Cisco SPA508G|        197|         3|2025-06-08|        591|
|Corporate|United States|   East|Cisco SPA508G|        197|         3|2025-06-08|        591|
| Consumer|United States|   East|Cisco SPA508G|         39|         1|2025-06-08|         39|
| Consumer|United States|   West|Cisco SPA508G|        211|         4|2025-06-08|        844|
| Consumer|United States|Central|Cisco SPA508G|        263|         4|2025-06-08|       1052|
+---------+-------------+-------+-------------+-----------+----------+----------+-----------+



In [56]:
#Exportar "linha_max" para um csv
df1_filtrado.toPandas().to_excel(f'Contem_Cisco_{hoje}.xlsx')
print(f"Dados exportados com êxito 'Contem_Cisco_{hoje}.xlsx'")

Dados exportados com êxito 'Contem_Cisco_2025-06-08.xlsx'


In [66]:
df.show()

+--------------+-----------+----------+-----------+-------------+-------+---------------+---------------+--------------------+-----------+----------+
|     ID_Pedido|Data_Pedido|ID_Cliente|   Segmento|         Pais| Regiao|     ID_Produto|      Categoria|        Nome_Produto|Valor_Venda|Quantidade|
+--------------+-----------+----------+-----------+-------------+-------+---------------+---------------+--------------------+-----------+----------+
|CA-2016-152156| 2016-11-08|  CG-12520|   Consumer|United States|  South|FUR-BO-10001798|      Furniture|Bush Somerset Col...|     261.96|      null|
|CA-2016-152156| 2016-11-08|  CG-12520|   Consumer|United States|  South|FUR-CH-10000454|      Furniture|Hon Deluxe Fabric...|     731.94|      null|
|CA-2016-138688| 2016-06-12|  DV-13045|  Corporate|United States|   West|OFF-LA-10000240|Office Supplies|Self-Adhesive Add...|      14.62|         2|
|US-2015-108966| 2015-10-11|  SO-20335|   Consumer|United States|  South|FUR-TA-10000577|      Furni

In [75]:
from pyspark.sql.functions import col, current_date, date_sub
df_filtro_data = df.filter(col("Data_Pedido") >= date_sub(current_date(), 30))

In [76]:
df_filtro_data.show()

+---------+-----------+----------+--------+----+------+----------+---------+------------+-----------+----------+
|ID_Pedido|Data_Pedido|ID_Cliente|Segmento|Pais|Regiao|ID_Produto|Categoria|Nome_Produto|Valor_Venda|Quantidade|
+---------+-----------+----------+--------+----+------+----------+---------+------------+-----------+----------+
+---------+-----------+----------+--------+----+------+----------+---------+------------+-----------+----------+

